## 文本相似度

文本数据是非结构化的和高噪声的.在执行文本分类时,拥有标记合理的训练数据和有监督学习大有裨益.

但是,文档聚类是一个无监督的学习过程,我们将尝试通过让机器学习各种各样的文本文档及其特征,相似

度以及它们之间的差异,来将文档分割和分类为单独的类别.

本章将重点介绍与文本相似度,距离度量和无监督ML算法相关的几个概念,以回答以下问题:

- 如何衡量文档之间的相似度?
- 如何使用距离测量值称为度量?
- 什么时候距离测量值称为度量?
- 如何聚类或组合类似的文档?
- 可以可视化文档聚类吗?

### 相似度测量
相似度或距离测量值通常是用来衡量两个实体之间的接近程度的,其中实体可以是任何文本形式,例如文档,句子甚至是短语.

实体之间的相似程度由两个主要因素决定:
- 实体的固有属性或特征
- 测量公式及其特性

### 无监督的机器学习算法
无监督的机器学习算法属于ML算法系列,从数据的各种属性和特征中发现其中潜在的,隐藏的结构和模式.

## 文本规范化

In [4]:
import nltk
import re

In [3]:
#更新停用词列表
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr','mrs','come','go','get','tell','listen','one','two','three','four',
                                'five','six','seven','eight','nine','zero','join','find','make','say','ask',
                                'tell','see','try','back','also']

In [5]:
#使用正则表达式从文本主体中提取文本标识
def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [6]:
def normalize_corpus(corpus, lemmatize=True,
                    only_text_chars=False,
                    tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = html_parser.unescape(text) #html分析
        text = expand_contractions(text, CONTRACTION_MAP) #扩展缩写词
        if lemmatize:
            text = lemmatize_text(text) #词形还原
        else:
            text = text.lower()
        
        # 删除停用词及特殊字符
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        
        #提取文本标识
        if only_text_chars:
            text = keep_text_characters(text)
            
        if tokenize:
            text = tokenize_text(text) #分词
            normalized_corpus.append(text)
        else:
            normalized_corpus.append(text)
    return normalized_corpus

## 特征提取

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1,1), min_df=0.0, max_df=1.0):
    '''
    min_df:忽略文档频率低于该阈值的特征
    max_df:忽略文档频率高于该阈值的特征
    '''
    feature_type = feature_type.lower().strip()
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    else:
        raise Exception('Wrong feature type entered.Possible values: "binary","frequency","tfidf"')
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

## 文本相似度
文本相似度分析的主要目的是分析和测量两个文本实体彼此距离的远近.

文本相似度分析的目的大致分为以下两个方面:
- 词汇相似度:通过句法,结构和内容研究文本文档的内容,并根据这些参数测量其相似度
- 语义相似度:首先找出文档的语义,含义和上下文,然后找出它们彼此的距离.在这方面,依存语法和实体识别是很有用的工具

接下来,将主要介绍以下两个领域的文本相似度:
- 词项相似度: 在这里,将测量每个标识或单词之间的相似度
- 文档相似度: 在这里,将测量整个文本文档之间的相似度

## 词项相似度分析
从分析单独的单词标识相似度入手.我们可以使用词项相似度分析中的部分技术来纠正拼写错误的词项.进行词项相似度分析,

首先需要选择一种单词标识方法,然后指定一种距离度量.我们将使用如下单词表示方法:

- 字符向量化
- 字符袋(Bag of Character)向量化

In [8]:
import numpy as np

def vectorize_terms(terms):
    terms = [term.lower() for term in terms]
    terms = [np.array(list(term)) for term in terms]
    terms = [np.array([ord(char) for char in term]) for term in terms]
    
    return terms

In [9]:
from scipy.stats import itemfreq

def boc_term_vectors(word_list):
    word_list = [word.lower() for word in word_list]
    unique_chars = np.unique(np.hstack([list(word) for word in word_list]))
    word_list_term_counts = [{char: count for char, count in itemfreq(list(word))} for word in word_list]
    boc_vectors = [np.array([int(word_term_counts.get(char, 0)) for char in unique_chars])
                  for word_term_counts in word_list_term_counts]
    
    return list(unique_chars), boc_vectors

In [10]:
root = 'Believe'
term1 = 'beleive'
term2 = 'bargain'
term3 = 'Elephant'
terms = [root, term1, term2, term3]
#Character vectorization
vec_root, vec_term1, vec_term2, vec_term3 = vectorize_terms(terms)
#show vector representations
print '''
root:{}
term1:{}
term2:{}
term3:{}
'''.format(vec_root, vec_term1, vec_term2, vec_term3)


root:[ 98 101 108 105 101 118 101]
term1:[ 98 101 108 101 105 118 101]
term2:[ 98  97 114 103  97 105 110]
term3:[101 108 101 112 104  97 110 116]



In [12]:
# bag of characters vectorization
features,(boc_root, boc_term1, boc_term2, boc_term3) = boc_term_vectors(terms)
#show features and vector representations
print 'Features:', features
print '''
root:{}
term1:{}
term2:{}
term3:{}
'''.format(boc_root, boc_term1, boc_term2, boc_term3)

Features: ['a', 'b', 'e', 'g', 'h', 'i', 'l', 'n', 'p', 'r', 't', 'v']

root:[0 1 3 0 0 1 1 0 0 0 0 1]
term1:[0 1 3 0 0 1 1 0 0 0 0 1]
term2:[2 1 0 1 0 1 0 1 0 1 0 0]
term3:[1 0 2 0 1 0 1 1 1 0 1 0]



本节将介绍以下五个度量:
- 汉明距离(Hamming distance)
- 曼哈顿距离(Manhattan distance)
- 欧几里得距离(Euclidean distance)
- 莱文斯坦编辑距离(Levenshtein edit distance)
- 余弦距离(Cosine distance)和相似度

首先,设置一些必要的变量以存储根词项,其他词项以及它们的向量化表示.

In [13]:
root_term = root
root_vector = vec_root
root_boc_vector = boc_root

term1 = [term1, term2, term3]
vector_terms = [vec_term1, vec_term2, vec_term3]
boc_vector_terms = [boc_term1, boc_term2, boc_term3]

### 汉明距离
汉明距离是两个长度相等的字符串之间的测量距离.它的正式定义是两个长度相等的字符串之间互异字符或符号的位置的数量.

In [14]:
def hamming_distance(u,v,norm=False):
    if u.shape!=v.shape:
        raise ValueError('The vectors must have equal lengths.')
        
    return (u!=v).sum() if not norm else (u!=v).mean()

In [15]:
#compute Hamming distance
for term,vector_term in zip(terms, vector_terms):
    print 'Hamming distance between root: {} and term: {} is {}'.format(root_term, term, 
                                                                        hamming_distance(root_vector,vector_term,norm=False))
    

Hamming distance between root: Believe and term: Believe is 2
Hamming distance between root: Believe and term: beleive is 6


ValueError: The vectors must have equal lengths.

In [ ]:
# 